# Analysis of Discrimination in Resume Rankings

We analyze GPT's biases for picking the top-qualified candidates for the occupations used in our tests.

In [1]:
import json
import glob
from collections import Counter

from tqdm import tqdm

import pandas as pd
from IPython.display import display, HTML

In [2]:
# outputs
fn_ranking = '../data/output/performance_ranking.csv'
fn_ranking_graphics = '../data/output/resume_ranking_for_graphics.csv'

# inputs
date = "0908_ages" # when data was collected
features = ['standard', 'female', 'male', 'nationality_standard', 'nationality_swiss', 'ages']
models = ['gpt-3.5-turbo', 'gpt-4o', 'gpt-4o-mini']

model2fileslen = {}
model2files = {}

for feature in features:
    model2files[feature] = {}
    model2fileslen[feature] = {}
    for model in models:
        model2fileslen[feature][model] = len(glob.glob(f'../data/intermediary/resume_ranking/{model}/*/{feature}/*.json'))
        model2files[feature][model] = glob.glob(f'../data/intermediary/resume_ranking/{model}/*/{feature}/*.json')

print(model2fileslen)

{'standard': {'gpt-3.5-turbo': 4550, 'gpt-4o': 4550, 'gpt-4o-mini': 4550}, 'female': {'gpt-3.5-turbo': 4375, 'gpt-4o': 4375, 'gpt-4o-mini': 4375}, 'male': {'gpt-3.5-turbo': 4375, 'gpt-4o': 4375, 'gpt-4o-mini': 4375}, 'nationality_standard': {'gpt-3.5-turbo': 4550, 'gpt-4o': 4550, 'gpt-4o-mini': 4550}, 'nationality_swiss': {'gpt-3.5-turbo': 4550, 'gpt-4o': 4550, 'gpt-4o-mini': 4550}, 'ages': {'gpt-3.5-turbo': 4200, 'gpt-4o': 4200, 'gpt-4o-mini': 4200}}


In [3]:
jobs = ['Pflegefachperson', 'Software Engineer', 'Talent Acquisition Specialist', 'Lehrperson Primarschule', 'Sales Consultant Software', 'Kaufmann_Kauffrau', 'Detailhandelsfachfrau_mann']

In [4]:
data = []
for feature in ['standard', 'female', 'male', 'nationality_standard', 'nationality_swiss', 'ages']:
    print(f"{feature}")
    for model in ['gpt-3.5-turbo', 'gpt-4o-mini', 'gpt-4o']:
        for N_top in range(1, 1+1):
            topistop = 0
            files = model2files[feature][model]
            print(f"top {N_top} {model}")
            _c = 0
            _top_og = Counter()
            _top_gpt = Counter()

            for fn in files:
                records = json.load(open(fn))
                sentence = records['choices'][0]['message']['content'].lower()
                context = records['context']
                _job = context['job']
                real_order = context['default_order']
                if feature != 'ages':
                    real_order = [_.lower() for _ in real_order]
                    demo_order = context['demo_order']

                    name2len = {}
                    for name in real_order:
                        name2len[name] = len(sentence.split(name)[0])
                    name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                    gpt_order = list(name2len.keys())

                    name2race = dict(zip(real_order, demo_order))
                    gpt_race_order = [
                        name2race.get(_) for _ in gpt_order
                    ]

                    _top_og.update(demo_order[:N_top])
                    _top_gpt.update(gpt_race_order[:N_top])
                    _c += 1

                    # determine how often #1 is the same as natural order
                    if gpt_race_order[0] == demo_order[0]:
                        topistop += 1

                else:
                    demo_order = context['default_order']

                    age2len = {}
                    for age in real_order:
                        age_str = str(age)
                        age2len[age] = len(sentence.split(age_str)[0])
                    age2len = dict(sorted(age2len.items(), key=lambda item: item[1]))
                    gpt_order = list(age2len.keys())

                    _top_og.update(demo_order[:N_top])
                    _top_gpt.update(gpt_order[:N_top])
                    _c += 1

                    if gpt_order[0] == demo_order[0]:
                        topistop += 1
                
            print(f"{topistop / len(files)}")
        
            for job in jobs:
                top_og = Counter()
                top_gpt = Counter()
                c = 0
                for fn in files:
                    records = json.load(open(fn))
                    sentence = records['choices'][0]['message']['content'].lower()
                    context = records['context']
                    _job = context['job']
                    real_order = context['default_order']

                    if feature != 'ages':
                        real_order = [_.lower() for _ in real_order]
                        demo_order = context['demo_order']
                        
                        name2len = {}
                        for name in real_order:
                            name2len[name] = len(sentence.split(name)[0])
                        name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                        gpt_order = list(name2len.keys())
                    
                        name2race = dict(zip(real_order, demo_order))
                        gpt_race_order = [
                            name2race.get(_) for _ in gpt_order
                        ]
                    
                        if _job == job:
                            top_og.update(demo_order[:N_top])
                            top_gpt.update(gpt_race_order[:N_top])
                            c += 1 

                    else:
                        demo_order = context['default_order']

                        age2len = {}
                        for age in real_order:
                            age_str = str(age)
                            age2len[age] = len(sentence.split(age_str)[0])
                        age2len = dict(sorted(age2len.items(), key=lambda item: item[1]))
                        gpt_order = list(age2len.keys())

                        if _job == job:
                            top_og.update(demo_order[:N_top])
                            top_gpt.update(gpt_order[:N_top])
                            c += 1

                print(job)
                if feature != 'ages':
                    df = pd.DataFrame(top_gpt.most_common(), columns=['demo', 'top'])
                    df_og = pd.DataFrame(top_og.most_common(), columns=['demo', 'top_og'])            
                    df = df.merge(df_og, on='demo')
                else:
                    df = pd.DataFrame(top_gpt.most_common(), columns=['age', 'top'])
                    df_og = pd.DataFrame(top_og.most_common(), columns=['age', 'top_og'])
                    df = df.merge(df_og, how="right", on='age')
                    df.fillna(0, inplace=True)
                    df = df.astype({'top':'int'})

                # Calculate selection rate and disparate impact ratio
                df['selection_rate'] = df['top'] / c
                df['disparate_impact_ratio'] = df['selection_rate'] / df['selection_rate'].max()

                # Optionally used for balancing data in notebook
                #df['to_collect'] = 20 - df['top_og']
                #df['to_collect'] = 125 - df['top_og']
                #df['to_collect'] = 65 - df['top_og']
                #df['to_collect'] = 75 - df['top_og']
                
                # Display the results for the current job
                display(HTML(df.sort_values(by='disparate_impact_ratio', ascending=True).reset_index(drop=1).to_html()))
                    
                # Add job and model details
                df['job'] = job
                df['model'] = model
                df['feature'] = feature
                df['rank'] = N_top

                # Append the results to the final data list
                data.extend(df.to_dict(orient='records'))

standard
top 1 gpt-3.5-turbo
0.44043956043956045
Pflegefachperson


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,Portugese_M,44,65,0.067692,0.500000
1,Swiss_M,54,65,0.083077,0.613636
2,Kosovo_M,57,65,0.087692,0.647727
3,Turkish_M,60,65,0.092308,0.681818
4,Turkish_W,65,65,0.100000,0.738636
5,Serbian_M,68,65,0.104615,0.772727
6,Kosovo_W,69,65,0.106154,0.784091
7,Swiss_W,72,65,0.110769,0.818182
8,Portugese_W,73,65,0.112308,0.829545
9,Serbian_W,88,65,0.135385,1.000000


Software Engineer


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,Portugese_M,39,65,0.060000,0.325000
1,Swiss_M,49,65,0.075385,0.408333
2,Kosovo_M,50,65,0.076923,0.416667
3,Portugese_W,58,65,0.089231,0.483333
4,Turkish_M,59,65,0.090769,0.491667
5,Swiss_W,61,65,0.093846,0.508333
6,Turkish_W,63,65,0.096923,0.525000
7,Kosovo_W,65,65,0.100000,0.541667
8,Serbian_M,86,65,0.132308,0.716667
9,Serbian_W,120,65,0.184615,1.000000


Talent Acquisition Specialist


...

,age,top,top_og,selection_rate,disparate_impact_ratio
0,60,10,75,0.016667,0.059524
1,55,27,75,0.045000,0.160714
2,25,38,75,0.063333,0.226190
3,50,42,75,0.070000,0.250000
4,45,89,75,0.148333,0.529762
5,40,113,75,0.188333,0.672619
6,30,113,75,0.188333,0.672619
7,35,168,75,0.280000,1.000000


Kaufmann_Kauffrau


,age,top,top_og,selection_rate,disparate_impact_ratio
0,60,2,75,0.003333,0.011696
1,55,17,75,0.028333,0.099415
2,50,19,75,0.031667,0.111111
3,45,48,75,0.080000,0.280702
4,40,54,75,0.090000,0.315789
5,35,134,75,0.223333,0.783626
6,25,155,75,0.258333,0.906433
7,30,171,75,0.285000,1.000000


Detailhandelsfachfrau_mann


,age,top,top_og,selection_rate,disparate_impact_ratio
0,60,3,75,0.005000,0.013274
1,55,8,75,0.013333,0.035398
2,50,10,75,0.016667,0.044248
3,45,20,75,0.033333,0.088496
4,40,25,75,0.041667,0.110619
5,35,125,75,0.208333,0.553097
6,30,183,75,0.305000,0.809735
7,25,226,75,0.376667,1.000000


Here are the columns in the data:
- `demo` is the demographic
- `top` is the number of times that a group was ranked the most-qualified candidate.
- `top_og` is how often the group was shown to GPT as the first name in the list of resumes.
- `selection_rate` is the frequency in which the group was ranked in the top.
- `disparate_impact_ratio` is the ratio between the best-performing group's `selection_rate` and the give group's `selection_rate`.

In [5]:
results = pd.DataFrame(data)

In [32]:
results.to_csv(fn_ranking, index=False)

## Data for visualization

Producing granular data.

In [6]:
data_clean = []
for feature in ['standard', 'female', 'male', 'nationality_standard', 'nationality_swiss', 'ages']:
    for model in ['gpt-3.5-turbo', 'gpt-4o-mini', 'gpt-4o']:
        files = model2files[feature][model]
        for fn in files:
            records = json.load(open(fn))
            sentence = records['choices'][0]['message']['content'].lower()
            context = records['context']
            _job = context['job']
            real_order = context['default_order']
            
            if feature != 'ages':
                real_order = [_.lower() for _ in real_order]
                demo_order = context['demo_order']

                name2len = {}
                for name in real_order:
                    name2len[name] = len(sentence.split(name)[0])
                name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                gpt_order = list(name2len.keys())
            
                name2race = dict(zip(real_order, demo_order))
                gpt_race_order = [
                    name2race.get(_) for _ in gpt_order
                ]
                data_clean.append({
                    "job" : _job,
                    "default_order_names" : real_order,
                    "default_order_demo" : demo_order, 
                    "gpt_ranking_names": gpt_order,
                    "gpt_ranking_demo": gpt_race_order,
                    "name2demo": name2race,
                    'model': model,
                    'feature': feature,
                    'fn' : fn
                })

            else:
                demo_order = context['default_order']

                age2len = {}
                for age in real_order:
                    age_str = str(age)
                    age2len[age] = len(sentence.split(age_str)[0])
                age2len = dict(sorted(age2len.items(), key=lambda item: item[1]))
                gpt_order = list(age2len.keys())
            
                data_clean.append({
                    "job" : _job,
                    "default_order_age" : real_order,
                    "gpt_ranking_age": gpt_order,
                    'model': model,
                    'feature': feature,
                    'fn' : fn
                })

In [38]:
pd.DataFrame(data_clean).to_csv(fn_ranking_graphics)

Making aggregate top and bottom-ranked.

In [7]:
data = []
for feature in ['standard', 'female', 'male', 'nationality_standard', 'nationality_swiss', 'ages']:
    for model in ['gpt-3.5-turbo', 'gpt-4o-mini', 'gpt-4o']:
        for N_top in [0, -1]:
            topistop = 0
            files = model2files[feature][model]
            print(f"top {N_top} {model}")
            _c = 0
            _top_og = Counter()
            _top_gpt = Counter()
            for job in jobs:
                top_og = Counter()
                top_gpt = Counter()
                c = 0
                for fn in files:
                    records = json.load(open(fn))
                    sentence = records['choices'][0]['message']['content'].lower()
                    context = records['context']
                    _job = context['job']
                    real_order = context['default_order']

                    if feature != 'ages':
                        real_order = [_.lower() for _ in real_order]
                        demo_order = context['demo_order']
                        
                        name2len = {}
                        for name in real_order:
                            name2len[name] = len(sentence.split(name)[0])
                        name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                        gpt_order = list(name2len.keys())
                    
                        name2race = dict(zip(real_order, demo_order))
                        gpt_race_order = [
                            name2race.get(_) for _ in gpt_order
                        ]
                    
                        if _job == job:
                            top_og.update([demo_order[N_top]])
                            top_gpt.update([gpt_race_order[N_top]])
                            c += 1 

                    else:
                        demo_order = context['default_order']

                        age2len = {}
                        for age in real_order:
                            age_str = str(age)
                            age2len[age] = len(sentence.split(age_str)[0])
                        age2len = dict(sorted(age2len.items(), key=lambda item: item[1]))
                        gpt_order = list(age2len.keys())

                        if _job == job:
                            top_og.update([demo_order[N_top]])
                            top_gpt.update([gpt_order[N_top]])
                            c += 1
                    
                # print 
                print(job)
                if feature != 'ages':
                    df = pd.DataFrame(top_gpt.most_common(), columns=['demo', 'top'])
                    df_og = pd.DataFrame(top_og.most_common(), columns=['demo', 'top_og'])            
                    df = df.merge(df_og, on='demo')
                else:
                    df = pd.DataFrame(top_gpt.most_common(), columns=['age', 'top'])
                    df_og = pd.DataFrame(top_og.most_common(), columns=['age', 'top_og'])
                    df = df.merge(df_og, how="right", on='age')
                    df.fillna(0, inplace=True)
                    df = df.astype({'top':'int'})

                df['selection_rate'] = df['top'] / c
                df['disparate_impact_ratio'] = df['selection_rate'] / df['selection_rate'].max()

                display(HTML(df.sort_values(by='disparate_impact_ratio', ascending=True).reset_index(drop=1).to_html()))
                df['job'] = job
                df['model'] = model
                df['feature'] = feature
                df['rank'] = N_top
        
                data.extend(df.to_dict(orient='records'))

top 0 gpt-3.5-turbo
Pflegefachperson


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,Portugese_M,44,65,0.067692,0.500000
1,Swiss_M,54,65,0.083077,0.613636
2,Kosovo_M,57,65,0.087692,0.647727
3,Turkish_M,60,65,0.092308,0.681818
4,Turkish_W,65,65,0.100000,0.738636
5,Serbian_M,68,65,0.104615,0.772727
6,Kosovo_W,69,65,0.106154,0.784091
7,Swiss_W,72,65,0.110769,0.818182
8,Portugese_W,73,65,0.112308,0.829545
9,Serbian_W,88,65,0.135385,1.000000


Software Engineer


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,Portugese_M,39,65,0.060000,0.325000
1,Swiss_M,49,65,0.075385,0.408333
2,Kosovo_M,50,65,0.076923,0.416667
3,Portugese_W,58,65,0.089231,0.483333
4,Turkish_M,59,65,0.090769,0.491667
5,Swiss_W,61,65,0.093846,0.508333
6,Turkish_W,63,65,0.096923,0.525000
7,Kosovo_W,65,65,0.100000,0.541667
8,Serbian_M,86,65,0.132308,0.716667
9,Serbian_W,120,65,0.184615,1.000000


Talent Acquisition Specialist


...

,age,top,top_og,selection_rate,disparate_impact_ratio
0,35,8,76,0.013333,0.023739
1,50,10,74,0.016667,0.029674
2,30,11,72,0.018333,0.032641
3,45,12,85,0.020000,0.035608
4,40,18,91,0.030000,0.053412
5,55,30,71,0.050000,0.089021
6,60,174,66,0.290000,0.516320
7,25,337,65,0.561667,1.000000


Kaufmann_Kauffrau


,age,top,top_og,selection_rate,disparate_impact_ratio
0,50,9,74,0.015000,0.023077
1,45,10,85,0.016667,0.025641
2,30,13,72,0.021667,0.033333
3,35,14,76,0.023333,0.035897
4,40,21,91,0.035000,0.053846
5,55,25,71,0.041667,0.064103
6,25,118,65,0.196667,0.302564
7,60,390,66,0.650000,1.000000


Detailhandelsfachfrau_mann


,age,top,top_og,selection_rate,disparate_impact_ratio
0,35,8,76,0.013333,0.020672
1,30,15,72,0.025000,0.038760
2,40,21,91,0.035000,0.054264
3,45,25,85,0.041667,0.064599
4,50,27,74,0.045000,0.069767
5,55,42,71,0.070000,0.108527
6,25,75,65,0.125000,0.193798
7,60,387,66,0.645000,1.000000


In [8]:
df = pd.DataFrame(data)

In [9]:
df.head(2)

,demo,top,top_og,selection_rate,disparate_impact_ratio,job,model,feature,rank,age
0,Serbian_W,88,65,0.135385,1.000000,Pflegefachperson,gpt-3.5-turbo,standard,0,NaN
1,Portugese_W,73,65,0.112308,0.829545,Pflegefachperson,gpt-3.5-turbo,standard,0,NaN


In [15]:
df_merged = df[df['rank'] == 0].merge(
    df[df['rank'] == -1][['demo', 'selection_rate', 'job', 'model', 'feature']],
    on=['demo', 'job', 'model', 'feature'],
    suffixes=['', '_bottom']
)
df_merged_ages = df[df['rank'] == 0].merge(
    df[df['rank'] == -1][['age', 'selection_rate', 'job', 'model', 'feature']],
    on=['age', 'job', 'model', 'feature'],
    suffixes=['', '_bottom']
)


In [16]:
df_merged['race/ethnicity'] = df_merged['demo'].str.split('_').str.get(0)
df_merged['sex'] = df_merged['demo'].str.split('_').str.get(1)

In [17]:
df_merged = df_merged[['feature', 'job','race/ethnicity', 'sex', 
                       'selection_rate', 'disparate_impact_ratio', 'selection_rate_bottom', 'model']]

df_merged_ages = df_merged_ages[['feature', 'job','age', 
                       'selection_rate', 'disparate_impact_ratio', 'selection_rate_bottom', 'model']]

In [18]:
df_merged.columns = ['feature', 'job', 'race/ethnicity', 'sex', 'top_ranked_perc', 'top_impact_ratio',
       'bottom_ranked_perc', 'model']

df_merged_ages.columns = ['feature', 'job', 'age', 'top_ranked_perc', 'top_impact_ratio',
       'bottom_ranked_perc', 'model']

In [19]:
df_merged.to_csv('../data/output/graphics_bw_performance_ranking.csv', index=False)

df_merged_ages.to_csv('../data/output/graphics_bw_performance_ranking_ages.csv', index=False)